コードを実行する前に，Google Drive のルートに `NovelAI` フォルダを作成しておく．

また，上の「ランタイム」から「ランタイムのタイプを変更」→「ハードウェア アクセラレータ」を `GPU` に変更する．

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
!pip install albumentations==0.4.3
!pip install diffusers
!pip install opencv-python==4.1.2.30
!pip install pudb==2019.2
!pip install imageio==2.9.0
!pip install imageio-ffmpeg==0.4.2
!pip install pytorch-lightning
!pip install omegaconf==2.1.1
!pip install test-tube>=0.7.5
!pip install streamlit>=0.73.1
!pip install einops==0.3.0
!pip install torch-fidelity==0.3.0
!pip install transformers==4.19.2
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install invisible-watermark

以下は Novel AI のモデルをダウンロードするためのコードなので，実行は初回のみでよい．

=== ここから ===

In [ ]:
!apt install python3-libtorrent
!pip install libtorrent

In [ ]:
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

params = {"save_path": "/content/drive/MyDrive/NovelAI"}

magnet_link = "magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce"

downloads.append(
    lt.add_magnet_uri(ses, magnet_link, params)
)


In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


ダウンロードが完了していても，Google Drive へアップロードが完全に終わるまで時間がかかるから，コーヒーでも嗜んで優雅に待とう．

=== ここまで ===

In [ ]:
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui

以下を変更

stable-diffusion-webui/modules/safety.py
```
# x_checked_image, has_nsfw_concept = check_safety(x_samples_ddim_numpy)
x_checked_image = x_samples_ddim_numpy
```

In [4]:
novelai_path = "/content/drive/MyDrive/NovelAI/novelaileak/"
!cp $novelai_path/stableckpt/animefull-final-pruned/model.ckpt models/Stable-diffusion/final-pruned.ckpt
!cp $novelai_path/stableckpt/animevae.pt models/Stable-diffusion/final-pruned.vae.pt
!mkdir models/hypernetworks
!cp $novelai_path/stableckpt/modules/modules/* models/hypernetworks/

以下を実行すると，サーバが起動する．表示された public URL にアクセスし，ユーザ名 : `user`，パスワード : `user` でログインする．

In [ ]:
!COMMANDLINE_ARGS="--share --gradio-debug --gradio-auth user:user" REQS_FILE="requirements.txt" python launch.py